# Segmenting Toronto Neighborhoods with K-Means Clustering

First we have to input the necessary libraries

In [15]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import folium
import pgeocode
print('Libraries imported.')

Libraries imported.


Check statues code to make sure we can scrape the website

In [2]:
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


Use Beautiful Soup to get the html of the dataframe

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
torontoHTML=soup.find('table',{'class':"wikitable"})

Read the html into a pandas dataframe and drop rows where the Borough is not assigned

In [4]:
torontoTable =pd.DataFrame(pd.read_html(str(torontoHTML))[0])
torontoTable.drop(torontoTable[torontoTable['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)

Check if there are any rows where the neighbourhood is not assigned or any equivalent

In [5]:
torontoTable['Neighbourhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
       'Fairview, H

It doesn't seem like there are any. Now we show the DataFrame

In [6]:
torontoTable.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Find the shape of the DataFrame

In [25]:
torontoTable.shape

(103, 3)

Let's see if geocoder will work

In [18]:
i = 0
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    i+=1
    print('iteration ' + str(i))
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48


KeyboardInterrupt: 

I don't think it will, so let's read the csv provided and set the index to the Postal Code to make joining the Dataframes easier

In [27]:
coordinates_df = pd.read_csv('Geospatial_Coordinates (1).csv')
coordinates_df.set_index('Postal Code', inplace = True)

Now let's join the two dataframes and see the result

In [30]:
torontoTable = torontoTable.join(coordinates_df, on = 'Postal Code', how = 'left')

ValueError: columns overlap but no suffix specified: Index(['Latitude', 'Longitude'], dtype='object')

In [31]:
torontoTable.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
